# Logging huggingface for auth 

In [ ]:
# Check requirements
# !pip3 install -q -r requirements.txt
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os

notebook_directory = os.path.dirname(os.path.abspath("__file__"))
directory = os.path.join(notebook_directory, "generation_result")
if not os.path.exists(directory):
	os.makedirs(directory)


# Import needed library and model

In [1]:
!pip3 install -q -r requirements.txt
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch, accelerate

# Models path
codellama_model = 'codellama/CodeLlama-7b-Instruct-hf'

# Adapters path
codellama_adapter = 'AIRLab-POLIMI/codellama-7b-instruct-hf-btgenbot-adapter'

# Postprocess function

In [2]:
import re
def extract_behavior_tree(result):
    # Define regular expression pattern to extract the behavior tree from the complete output
    pattern = r'<root .*?</root>'
    matches = re.findall(pattern, result, re.DOTALL)

    if matches:
        final_tree = matches[-1]
        return final_tree
    else:
        print("No start root of behavior tree please check the output")
    return None 



# Load pretrained codellama_model 

In [3]:
# Select here the model and the corresponding adapter
cache_dir = '/home/arg/BTGenBot/model'
model_id = codellama_model
adapter_id = codellama_adapter

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path = model_id,
    cache_dir=cache_dir
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = model_id,
    quantization_config = quantization_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
    cache_dir=cache_dir
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load Task description, similar example task

## Sample task of paper

In [4]:
context = "<<SYS>> You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.\n <</SYS>>"
task = """The behavior tree first checks if there is any object in view. If there is an object, it simultaneously executes control actions for linear x and linear y directions. If no object is in view, the robot will then proceed to explore a pattern block."""

# One-shot example
example_task = """The behavior tree represents a robot's navigation system with arm activity. The robot must visit the location "Station A", then follow the aruco with ID=7. The only available actions that must be used in the behavior tree are: "MoveTo", "FollowAruco"."""
example_output = """
<root main_tree_to_execute = "MainTree" >
    <BehaviorTree ID="MainTree">
        <Sequence>
            <MoveTo location="Station A"/>
            <FollowAruco id="7"/>
        </Sequence>
    </BehaviorTree>
</root>
"""

## Example for locobot navigation

In [ ]:
context = "<<SYS>> You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.\n <</SYS>>"
task = """The behavior tree represents a robot's navigation between different workspace for checking the area. 
The order of action is the following. 
Start from "start_position" and move to "workstation_A", then "workstation_B" and finally "workstation_C". After go through all workspace move back to "start_position", for security if the robot see the red light it need to "stop" immediately. 
The only available actions that must be used in the behavior tree are: "MoveTo", "Pick", "Press", "Drop", "Red_light_in_view", "stop"."""

example_task = """The behavior tree represents a robot's navigation between different workspace for checking the area. 
The order of action is the following. 
Start from "start_position" and move to "workstation_A", then "workstation_B" and finally "workstation_C". After go through all workspace move back to "start_position", for checking the area is clear, if the robot see the red ball, it need to "pick" the red ball up.
The only available actions that must be used in the behavior tree are: "MoveTo", "Pick", "Press", "Drop","Red_ball_in_view"."""
example_output = """
<root main_tree_to_execute = "MainTree" >
    <BehaviorTree ID="MainTree">
        <fallback>
            <Sequence>
                <Red_ball_in_view/>
                <Pick item="Red_ball"/>
            <Sequence>   
            <Sequence>
                <MoveTo location="start_position"/>
                <MoveTo location="workstattion_A"/>
                <MoveTo location="workstattion_B"/>
                <MoveTo location="workstattion_C"/>
                <MoveTo location="start_position"/>
            </Sequence>
        </fallback>
        
    </BehaviorTree>
</root>
"""

# Inference of pretrain base model

In [5]:
eval_prompt = context + "[INST]" + task + "[/INST]"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

In [6]:
print(eval_prompt)

<<SYS>> You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.
 <</SYS>>[INST]The behavior tree first checks if there is any object in view. If there is an object, it simultaneously executes control actions for linear x and linear y directions. If no object is in view, the robot will then proceed to explore a pattern block.[/INST]


In [ ]:
base_model.eval()
with torch.no_grad():
    result = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=3000)[0], skip_special_tokens=True)
    print("Generation result of base model")
    print(result)
    print("--------------------------------------------------------------------")
    xml_tree_base  = extract_behavior_tree(result)
    print(xml_tree_base)
    print("--------------------------------------------------------------------")

In [ ]:
response_base = os.path.join(directory, "response_base.txt")
with open(response_base, "w+") as file:
    file.write(result)
    
xml_base_filename = os.path.join(directory, "response_base_xml_tree.txt")
with open(xml_base_filename, "w+") as file:
    file.write(xml_tree_base)

# Inference of finetune model

In [7]:
# Load fine-tuned model
finetuned_model = PeftModel.from_pretrained(base_model, adapter_id)
finetuned_model = finetuned_model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/80.1M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:83: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [8]:
finetuned_model.eval()
with torch.no_grad():
    fine_tune_result = tokenizer.decode(finetuned_model.generate(**model_input, max_new_tokens=3000)[0], skip_special_tokens=True)
    print("Generation result of finetune model")
    print(fine_tune_result)
    print("--------------------------------------------------------------------")
    xml_tree_finetune = extract_behavior_tree(fine_tune_result)
    print(xml_tree_finetune)
    print("--------------------------------------------------------------------")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generation result of finetune model
<<SYS>> You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.
 <</SYS>>[INST]The behavior tree first checks if there is any object in view. If there is an object, it simultaneously executes control actions for linear x and linear y directions. If no object is in view, the robot will then proceed to explore a pattern block.[/INST] <root main_tree_to_execute="BehaviorTree">
    <BehaviorTree ID="BehaviorTree">
        <Sequence>
            <Fallback>
                <Sequence>
                    <Condition ID="HasObjectInView"/>
                    <Sequence>
                        <Action ID="RobotControlAction" linear_x="1" linear_y="1"/>
                    </Sequence>
                </Sequence>
            </Fallback>
            <Action ID="RobotExploreBlock" pattern_block="1"/>
        </Sequence>
    </BehaviorTree>
</root>

----------------------------------

In [ ]:
response_finetune = os.path.join(directory, "response_finetune.txt")
with open(response_finetune, "w+") as file:
    file.write(fine_tune_result)
    
xml_fintune_filename = os.path.join(directory, "response_finetune_xml_tree.txt")
with open(xml_fintune_filename, "w+") as file:
    file.write(xml_tree_finetune)

# Turn XML behavior tree to CMU symbolic tree version 